In [1]:
%pip install ortools

使用colab，因为本地在导入ortools包时出现了问题

In [2]:
import ortools

In [3]:
from ortools.linear_solver import pywraplp

将文件挂载到colab上

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [5]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/")

导入需要的包

In [6]:
import pandas as pd 

将family_id作为index，否则之后在计算preference cost的时候会有index报错

In [7]:
data = pd.read_csv('family_data.csv', index_col = 'family_id')

In [8]:
data

,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
family_id,,,,,,,,,,,
0,52,38,12,82,33,75,64,76,10,28,4
1,26,4,82,5,11,47,38,6,66,61,4
2,100,54,25,12,27,82,10,89,80,33,3
3,2,95,1,96,32,6,40,31,9,59,2
4,53,1,47,93,26,3,46,16,42,39,4
...,...,...,...,...,...,...,...,...,...,...,...
4995,16,1,66,33,18,70,56,46,86,60,4
4996,88,66,20,17,26,54,81,91,59,48,2
4997,32,66,54,17,27,21,74,81,3,7,6


In [9]:
#数据预处理

In [10]:
def get_penalty (n, choice):
    penalty = None
    if choice == 0:
        penalty = 0
    if choice == 1: 
        penalty = 50 
    if choice == 2: 
        penalty = 50 + 9*n
    if choice == 3: 
        penalty = 100 + 9*n
    if choice == 4: 
        penalty = 200 + 9*n
    if choice == 5: 
        penalty = 200 + 18*n
    if choice == 6: 
        penalty = 300 + 18*n
    if choice == 7: 
        penalty = 300 + 36*n
    if choice == 8: 
        penalty = 400 + 36*n
    if choice == 9: 
        penalty = 500 + (36+199)*n
    if choice > 9: 
        penalty = 500 + (36+398)*n
    return penalty


In [11]:
#安排的天数
N_days = 100 
#家庭个数
N_family = 5000 
#最小承载量
Min_occupancy = 125
#最大承载量
Max_occupancy = 300

In [12]:
import numpy as np

In [13]:
pcost_mat = np.full(shape =(N_family, 100), fill_value= 999999)
pcost_mat.shape

(5000, 100)

In [14]:
for f in range(N_family):
    # 家庭成员数
    f_num = data.loc[f, 'n_people']
    #print(f_num)
    # 对于第f个家庭，初始化pcost = other choice下的penalty
    pcost_mat[f, :] = get_penalty(f_num, 10)
    
    #计算choice0-9的penalty
    for choice in range(10):
        temp  = data.loc[f][choice] #choice的天数
        #print('temp: ',temp)
        penalty = get_penalty(f_num, choice)
        pcost_mat[f, temp-1] = penalty

In [15]:
pcost_mat

array([[2236, 2236, 2236, ..., 2236, 2236, 2236],
       [2236, 2236, 2236, ..., 2236, 2236, 2236],
       [1802, 1802, 1802, ..., 1802, 1802,    0],
       ...,
       [3104, 3104,  616, ..., 3104, 3104, 3104],
       [ 390, 2670, 2670, ..., 2670, 2670, 2670],
       [2236, 2236, 2236, ..., 2236, 2236, 2236]])

In [50]:
acost_mat = np.zeros(shape=(Max_occupancy+1,Max_occupancy+1),dtype=np.float64)

In [17]:
for i in range(Max_occupancy): #当天的安排人数
    for j in range(Max_occupancy): #前天的安排人数
        diff = np.abs(i-j)
        #防止负数出现，对下面方程的负数结果改为0
        acost_mat[i,j] = max(0,(i-125)/400*i**(0.5 + diff/50.0))
acost_mat

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [3.61426498e+15, 3.22526096e+15, 2.87812553e+15, ...,
        7.41048581e+00, 8.30427666e+00, 9.30586908e+00],
       [4.16316072e+15, 3.71482922e+15, 3.31477861e+15, ...,
        8.36716954e+00, 7.46610759e+00, 8.36716954e+00],
       [4.79555148e+15, 4.27883100e+15, 3.81778713e+15, ...,
        9.44825702e+00, 8.43020770e+00, 7.52185316e+00]])

In [18]:
family_size = data['n_people'].values
family_size

array([4, 4, 3, ..., 6, 5, 4])

In [19]:
desired = data.values[:,:-1]-1 
desired

array([[51, 37, 11, ..., 75,  9, 27],
       [25,  3, 81, ...,  5, 65, 60],
       [99, 53, 24, ..., 88, 79, 32],
       ...,
       [31, 65, 53, ..., 80,  2,  6],
       [66, 91,  3, ..., 11, 25, 69],
       [12, 10, 24, ..., 38, 17, 46]])

使用LP和MIP求解规划方案

In [20]:
def solveLP():
    #线性规划 优化器
    solver = pywraplp.Solver('AssignmentProblem',pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    
    x = {} #family_id 在第j天是否参观
    # 每天有哪些家庭
    candidates = [[] for x in range(N_days)]
    
    for i in range(N_family):
        for j in desired[i,:]: #family_id的choice
            candidates[j].append(i) #在第j天，有i个家庭参观
            # 定义决策变量 x[i,j] i代表famil_id j代表第j天参观
            x[i,j]= solver.BoolVar('x[%i,%i]'%(i,j))
    #每天参观的人数
    daily_occupancy = [solver.Sum([x[i,j] * family_size[i] for i in candidates[j]]) \
                for j in range(N_days)]
    
    #每个家庭，在10个choice中出现的总数
    family_presence = [solver.Sum(x[i,j] for j in desired[i,:]) \
              for i in range(N_family)]

    #定义目标函数 preference cost部分
    preference_cost = solver.Sum([pcost_mat[i,j] * x[i,j] for i in range(N_family) for j in desired[i,:]])
    
    # 满足preference cost最小
    solver.Minimize(preference_cost)

    # 约束条件
    for j in range(N_days-1):
      solver.Add(daily_occupancy[j] - daily_occupancy[j+1] <= 25)
      solver.Add(daily_occupancy[j+1] - daily_occupancy[j] <= 25)

    # 每个家庭都在10个choice中出现1次
    for i in range(N_family):
      solver.Add(family_presence[i]==1)
    # 每天访问人数约束
    for j in range(N_days):
      solver.Add(daily_occupancy[j] >= Min_occupancy)
      solver.Add(daily_occupancy[j] <= Max_occupancy)
    result = solver.Solve()
    # result
    temp = [(i,j,x[i,j].solution_value()) for i in range(N_family) \
      for j in desired[i,:] if x[i,j].solution_value() >0]
    #print(temp)
    print(solver.Objective().Value())
    # 得到参观日期的安排
    df = pd.DataFrame(temp, columns=['family_id','day','result'])
    return df

    print(x)

In [21]:
%%time
result = solveLP()

73702.31696428571
CPU times: user 15.7 s, sys: 114 ms, total: 15.8 s
Wall time: 15.8 s


In [22]:
result

,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5075,4995,15,1.0
5076,4996,87,1.0
5077,4997,31,1.0
5078,4998,91,1.0


In [23]:
# 设置阈值
THRS = 0.999
assigned_df = result[result.result > THRS]
assigned_df

,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5075,4995,15,1.0
5076,4996,87,1.0
5077,4997,31,1.0
5078,4998,91,1.0


In [24]:
# 没有安排的，不为0和1
unassigned_df = result[(result.result < THRS) & (result.result > 1-THRS)]
unassigned_df

,family_id,day,result
60,59,38,0.25
61,59,14,0.75
242,240,32,0.75
243,240,56,0.25
265,262,31,0.50
...,...,...,...
4990,4912,8,0.40
4992,4914,38,0.60
4993,4914,43,0.40
5040,4961,53,0.75


In [25]:
# 计算每天访问的人数
assigned_df['family_size'] = family_size[assigned_df.family_id]
assigned_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,family_id,day,result,family_size
0,0,51,1.0,4
1,1,25,1.0,4
2,2,99,1.0,3
3,3,1,1.0,2
4,4,52,1.0,4
...,...,...,...,...
5075,4995,15,1.0,4
5076,4996,87,1.0,2
5077,4997,31,1.0,6
5078,4998,91,1.0,5


In [27]:
# 按照day进行聚合
occupancy = assigned_df.groupby('day').family_size.sum().values
occupancy

array([285, 271, 294, 293, 263, 242, 223, 247, 273, 297, 288, 292, 275,
       250, 245, 272, 292, 292, 271, 248, 223, 244, 264, 291, 292, 296,
       273, 249, 234, 251, 278, 283, 252, 235, 205, 184, 202, 233, 253,
       226, 210, 183, 204, 229, 247, 281, 256, 223, 204, 198, 222, 248,
       256, 223, 208, 185, 173, 196, 219, 198, 174, 141, 124, 121, 149,
       170, 178, 160, 136, 123, 125, 135, 158, 185, 158, 134, 121, 125,
       142, 167, 186, 167, 138, 120, 128, 155, 174, 203, 177, 152, 130,
       122, 130, 158, 183, 158, 128, 125, 122, 124])

In [28]:
min_occupancy = np.array([max(0,Min_occupancy - x) for x in occupancy])
min_occupancy

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0,
       0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 1])

In [29]:
max_occupancy = np.array([Max_occupancy - x for x in occupancy])
max_occupancy

array([ 15,  29,   6,   7,  37,  58,  77,  53,  27,   3,  12,   8,  25,
        50,  55,  28,   8,   8,  29,  52,  77,  56,  36,   9,   8,   4,
        27,  51,  66,  49,  22,  17,  48,  65,  95, 116,  98,  67,  47,
        74,  90, 117,  96,  71,  53,  19,  44,  77,  96, 102,  78,  52,
        44,  77,  92, 115, 127, 104,  81, 102, 126, 159, 176, 179, 151,
       130, 122, 140, 164, 177, 175, 165, 142, 115, 142, 166, 179, 175,
       158, 133, 114, 133, 162, 180, 172, 145, 126,  97, 123, 148, 170,
       178, 170, 142, 117, 142, 172, 175, 178, 176])

In [31]:
#使用整数规划进行求解
def solveIP(families, min_occupancy, max_occupancy):
  #创建求解器
  solver = pywraplp.Solver('AssignmentProblem',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
  # 需要安排的家庭
  n_families = len(families)
  
  x = {}

  # 每天有哪些家庭
  candidates = [[] for x in range(N_days)]
    
  for i in families: #family_id
      for j in desired[i,:]: #family_id的choice
          candidates[j].append(i) #在第j天，有i个家庭参观
          # 定义决策变量 x[i,j] i代表famil_id j代表第j天参观
          x[i,j]= solver.BoolVar('x[%i,%i]'%(i,j))
    
  #每天参观的人数, x[i,j] = 0 或者1
  daily_occupancy = [solver.Sum([x[i,j] * family_size[i] for i in candidates[j]]) \
                for j in range(N_days)]
    
  #每个家庭，在10个choice中出现的总数
  family_presence = [solver.Sum(x[i,j] for j in desired[i,:]) \
              for i in families]

  #定义目标函数 preference cost部分
  preference_cost = solver.Sum([pcost_mat[i,j] * x[i,j] for i in families for j in desired[i,:]])
    
  # 满足preference cost最小
  solver.Minimize(preference_cost)

  # 每个家庭都在10个choice中出现1次
  for i in range(n_families):
    solver.Add(family_presence[i]==1)
  # 每天访问人数约束
  for j in range(N_days):
    solver.Add(daily_occupancy[j] >= min_occupancy[j])
    solver.Add(daily_occupancy[j] <= max_occupancy[j])
  result = solver.Solve()
  # result
  temp = [(i,j) for i in families \
    for j in desired[i,:] if x[i,j].solution_value() >0]
    
  # 计算出剩余家庭的安排
  df = pd.DataFrame(temp, columns=['family_id','day'])
  return df


In [33]:
#没有安排的family id
unassigned = unassigned_df.family_id.unique()

In [34]:
result = solveIP(unassigned,min_occupancy, max_occupancy)

In [35]:
result

,family_id,day
0,59,38
1,240,32
2,262,31
3,488,39
4,500,26
...,...,...
64,4850,4
65,4886,98
66,4912,17
67,4914,38


In [40]:
final_result = pd.concat((assigned_df[['family_id','day']], result)).sort_values('family_id')

In [41]:
final_result

,family_id,day
0,0,51
1,1,25
2,2,99
3,3,1
4,4,52
...,...,...
5075,4995,15
5076,4996,87
5077,4997,31
5078,4998,91


对结果进行评估

In [42]:
# 根据安排情况，计算安排的oreference cost
def pcost(prediction):
  daily_occupancy = np.zeros(N_days+1, dtype=np.int64)
  penalty = 0
  for (i,p) in enumerate(prediction):
    #计算家庭人数
    n = family_size[i]
    # 第i个家庭，p天访问时的cost
    penalty += pcost_mat[i,p]
    #计算当天的人数
    daily_occupancy[p] += n
  return penalty, daily_occupancy


In [51]:
#根据安排情况，计算安排的accounting cost
def acost(daily_occupancy):
  accounting_cost = 0
  num_out_of_range = 0
  for day in range(N_days):
    n_pl = daily_occupancy[day+1] #前一天
    n = daily_occupancy[day] #当天
    #如果超过承载范围，则设置out of range
    num_out_of_range += (n>Max_occupancy) or (n<Min_occupancy)
    #计算accouting cost
    accounting_cost += acost_mat[n, n_pl]
  return accounting_cost, num_out_of_range

In [44]:
#根据安排prediction
def cost_function(prediction):
  #基于prediction来计算preference cost和accounting cost
  penalty,daily_occupancy = pcost(prediction) #统计preference cost
  accounting_cost, num_out_of_range = acost(daily_occupancy) #根据每天承载数量，计算accounting cost
  final_score = penalty + accounting_cost + num_out_of_range * 99999999
  return final_score

In [46]:
prediction = final_result.day.values

In [52]:
print(cost_function(prediction))

72322.0


In [53]:
def save_result(pred, filename):
  result = pd.DataFrame(range(N_family), columns=['family_id'])
  result['assigned_day'] = pred +1
  result.to_csv(filename, index=False)
  print(filename + 'saved')
  return result

In [54]:
result = save_result(prediction,'action_result.csv')

action_result.csvsaved


In [55]:
# 寻找更好的方案

In [56]:
def find_better(pred):
  fobs = np.argsort(family_size)
  print(fobs)
  score = cost_function(pred)
  original_score = np.inf #打擂
  
  while score<original_score:
    original_score = score

    for family_id in fobs:
      for i in range(10):
        day = desired[family_id,i]
        #该family的原有日期 oldvalue
        oldvalue = pred[family_id]
        pred[family_id] = display
        new_score = cost_function(pred)
        #如果比原来分数小，更新choice
        if new_score < score:
          score = new_score
        else:
          pred[family_id] = oldvalue
      

new = prediction.copy()
find_better(prediction)

[2070 4520 4188 ... 1081 1099 3186]
